**Final Project Draft - CS598 DL4H in Spring 2023** 

  Jair Morais - jairv2@illinois.edu


Group ID: 210

Paper ID: 24

Install stop_words module:

In [ ]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=498262a88593b3c6644c9a0d1b75f68e22162ef83b31723a516825f1ceb794af
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


Main Declaration:

In [ ]:
import codecs
from collections import defaultdict
import csv
import string
import numpy as np
from stop_words import get_stop_words    # download stop words package from https://pypi.org/project/stop-words/


It is necessary to put the noteevents.csv and diagnoses_icd.csv on the root path to run the code.

The Noteevent table at MIMIC-III (this table re-
quire credential access to the physionet database),
the diagnosis icd table at MIMIC-III, it will be used
the demo version of that.

In [ ]:
stop_words = get_stop_words('english')
admidic=defaultdict(list)
count=0


with open('NOTEEVENTS2.csv', 'r') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
     for row in spamreader:
         dischargesummary = row[6].strip()
        
         if dischargesummary in "Discharge summary":
             admidic[row[2]].append(row[-1].replace('\n',' ').translate(str.maketrans('','',string.punctuation)).lower())
             count=count+1



In [ ]:
u=defaultdict(int)
for i in admidic:
    for jj in admidic[i]:
        line=jj.strip('\n').split()
        for j in line:
            u[j]=u[j]+1

u2=defaultdict(int)
for i in u:
        if i.isdigit()==False:
            if u[i]>10:
                if i not in stop_words:
                    u2[i]=u[i]
                    
u=[]   

file1=codecs.open('DIAGNOSES_ICD2.csv','r')
ad2c=defaultdict(list)
line=file1.readline()

while line:
    line=line.strip().split(',')

    if line[4][1:-1]!='':
        ad2c[line[2]].append("d_"+line[4][1:-1])
    
    line=file1.readline()

codeu=defaultdict(int)
for i in ad2c:
    for j in ad2c[i]:
        codeu[j]=codeu[j]+1



In [ ]:
cthre=0
fileo=codecs.open("combined_dataset",'w')

IDlist=np.load('IDlist.npy',encoding='bytes').astype(str)

for i in IDlist:
    if ad2c[i]!=[]:
        fileo.write('start! '+i+'\n')
        fileo.write('codes: ')
        tempc=[]
        for code in ad2c[i]:
            if codeu[code]>=cthre:
                if code[0:5] not in tempc:
                    tempc.append(code[0:5])
       
        for code in tempc:
            fileo.write(code+" ")
        fileo.write('\n')
        fileo.write('notes:\n')
      
        for line in admidic[i]:    
            thisline=line.strip('\n').split() 
            
            for j in thisline:
                
                if u2[j]!=0:
                    
                    fileo.write(j+" ")
            fileo.write('\n')
        fileo.write('end!\n')
fileo.close()

In [ ]:
import codecs
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


wikivocab={}
file1=codecs.open("wikipedia_knowledge",'r','utf-8')
line=file1.readline()
while line:
    if line[0:3]!='XXX':
        line=line.strip('\n')
        line=line.split()
        for i in line:
            wikivocab[i.lower()]=1
    line=file1.readline()

In [ ]:
notesvocab={}
filec=codecs.open("combined_dataset",'r','utf-8')

line=filec.readline()

while line:
    line=line.strip('\n')
    line=line.split()
    
    if line[0]=='codes:':
        line=filec.readline()
        line=line.strip('\n')
        line=line.split()
        
        if  line[0]=='notes:':
            
            line=filec.readline()
            
            while line!='end!\n':
                line=line.strip('\n')
                line=line.split()
                for word in line:
                    notesvocab[word]=1
                
                line=filec.readline()
                
            
    line=filec.readline()



a1=set(notesvocab)
a2=set(wikivocab)
a3=a1.intersection(a2)


In [ ]:
wikidocuments=[]
file2=codecs.open("wikipedia_knowledge",'r','utf-8')
line=file2.readline()
while line:
    if line[0:4]=='XXXd':
        tempf=[]
        line=file2.readline()
        while line[0:4]!='XXXe':
            line=line.strip('\n')
            line=line.split()
            for i in line:
                if i.lower() in a3:
                    tempf.append(i.lower())
            line=file2.readline()
        wikidocuments.append(tempf)
        
    line=file2.readline()


notesdocuments=[]
file3=codecs.open("combined_dataset",'r','utf-8')

line=file3.readline()

In [ ]:
while line:
    line=line.strip('\n')
    line=line.split()
    if line[0]=='codes:':
        line=file3.readline()
        line=line.strip('\n')
        line=line.split()
        
        if  line[0]=='notes:':
            tempf=[]
            line=file3.readline()
        
            while line!='end!\n':
                line=line.strip('\n')
                line=line.split()
                for word in line:
                    if word in a3:
                        tempf.append(word)
                
                line=file3.readline()
                
            
            notesdocuments.append(tempf)
    line=file3.readline()


In [ ]:

notesvocab={}
for i in notesdocuments:
    for j in i:
        if j.lower() not in notesvocab:
            notesvocab[j.lower()]=len(notesvocab)
notedata=[]
for i in notesdocuments:
    temp=''
    for j in i:
        temp=temp+j+" "
    notedata.append(temp)
    

wikidata=[]
for i in wikidocuments:
    temp=''
    for j in i:
        temp=temp+j+" "
    wikidata.append(temp)    
##########################################################
vect = CountVectorizer(min_df=1,vocabulary=notesvocab,binary=True)
binaryn = vect.fit_transform(notedata)
binaryn=binaryn.A
binaryn=np.array(binaryn,dtype=float)

vect2 = CountVectorizer(min_df=1,vocabulary=notesvocab,binary=True)
binaryk = vect2.fit_transform(wikidata)
binaryk=binaryk.A
binaryk=np.array(binaryk,dtype=float)


np.save('notevec',binaryn)
np.save('wikivec',binaryk)

In [ ]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
import codecs


##########################################################
wikivoc={}
codewiki=defaultdict(list)

file2=codecs.open("wikipedia_knowledge",'r','utf-8')
line=file2.readline()
count=0
while line:
    if line[0:4]=='XXXd':
        line=line.strip('\n')
        line=line.split()
        for i in line:
            if i[0:2]=='d_':
                codewiki[i].append(count)
                wikivoc[i]=1
        count=count+1
    line=file2.readline()

################### four codes have two wikidocuments, correct them
codewiki['d_072']=[214]
codewiki['d_698']=[125]
codewiki['d_305']=[250]
codewiki['d_386']=[219]

np.save('wikivoc',wikivoc)
##################################################
filec=codecs.open("combined_dataset",'r','utf-8')

line=filec.readline()

feature=[]
label=[]

while line:
    line=line.strip('\n')
    line=line.split()
    
    if line[0]=='codes:':
        temp=line[1:]
        label.append(temp)
        line=filec.readline()
        line=line.strip('\n')
        line=line.split()
        if  line[0]=='notes:':
            tempf=[]
            line=filec.readline()
           
            while line!='end!\n':
                line=line.strip('\n')
                line=line.split()
                tempf=tempf+line
                line=filec.readline()
            feature.append(tempf)
    line=filec.readline()


prevoc={}
for i in label:
    for j in i:
        if j not in prevoc:
            prevoc[j]=len(prevoc)

##################################
notevec=np.load('notevec.npy')
wikivec=np.load('wikivec.npy')
label_to_ix = {}
ix_to_label={}


for codes in label:
    for code in codes:
        if code not in label_to_ix:
            label_to_ix[code]=len(label_to_ix)
            ix_to_label[label_to_ix[code]]=code

tempwikivec=[]

for i in range(0,len(ix_to_label)):
    if ix_to_label[i] in wikivoc:
        temp=wikivec[codewiki[ix_to_label[i]][0]]
        tempwikivec.append(temp)
    else:
        tempwikivec.append([0.0]*wikivec.shape[1])
wikivec=np.array(tempwikivec)

####################################

data=[]
for i in range(0,len(feature)):
    data.append((feature[i], notevec[i], label[i]))
    
data=np.array(data)  

label_to_ix = {}
ix_to_label={}

for doc, note, codes in data:
    for code in codes:
        if code not in label_to_ix:
            if code in wikivoc:
                label_to_ix[code]=len(label_to_ix)
                ix_to_label[label_to_ix[code]]=code

np.save('label_to_ix',label_to_ix)
np.save('ix_to_label',ix_to_label)

training_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
training_data, val_data = train_test_split(training_data, test_size=0.125, random_state=42)

np.save('training_data',training_data)
np.save('test_data',test_data)
np.save('val_data',val_data)


word_to_ix = {}
ix_to_word={}
ix_to_word[0]='OUT'


for doc, note, codes in training_data:
    for word in doc:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)+1
            ix_to_word[word_to_ix[word]]=word  
    
np.save('word_to_ix',word_to_ix)
np.save('ix_to_word',ix_to_word)

newwikivec=[]
for i in range(0,len(ix_to_label)):
    newwikivec.append(wikivec[prevoc[ix_to_label[i]]])
newwikivec=np.array(newwikivec)
np.save('newwikivec',newwikivec)


/var/folders/1f/ddtsmdgd43dby3cx3r_v_xpr0000gn/T/ipykernel_67828/94965002.py:98: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data=np.array(data)


**KSI_CAML**

In [64]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import copy

##########################################################

label_to_ix=np.load('label_to_ix.npy',allow_pickle=True).item()
ix_to_label=np.load('ix_to_label.npy',allow_pickle=True)
training_data=np.load('training_data.npy',allow_pickle=True)
test_data=np.load('test_data.npy',allow_pickle=True)
val_data=np.load('val_data.npy',allow_pickle=True)
word_to_ix=np.load('word_to_ix.npy',allow_pickle=True).item()
ix_to_word=np.load('ix_to_word.npy',allow_pickle=True)
newwikivec=np.load('newwikivec.npy',allow_pickle=True)
wikivoc=np.load('wikivoc.npy',allow_pickle=True).item()

wikisize=newwikivec.shape[0]
rvocsize=newwikivec.shape[1]
wikivec=autograd.Variable(torch.FloatTensor(newwikivec))


batchsize=8




In [ ]:
def preprocessing(data):
    new_data=[]
    for i, note, j in data:
        templabel=[0.0]*len(label_to_ix)
        for jj in j:
            if jj in wikivoc:
                templabel[label_to_ix[jj]]=1.0
        templabel=np.array(templabel,dtype=float)
        new_data.append((i, note, templabel))
    #new_data=np.array(new_data)
    
    lenlist=[]
    for i in new_data:
        lenlist.append(len(i[0]))
    sortlen=sorted(range(len(lenlist)), key=lambda k: lenlist[k])  
    
    data_transform = []
    for j in sortlen:
        data_transform.append(new_data[j])
    new_data=data_transform #new_data[sortlen]
    
    batch_data=[]
    
    for start_ix in range(0, len(new_data)-batchsize+1, batchsize):
        thisblock=new_data[start_ix:start_ix+batchsize]
        mybsize= len(thisblock)
        numword=np.max([len(ii[0]) for ii in thisblock])
        main_matrix = np.zeros((mybsize, numword), dtype= np.int32)
        for i in range(main_matrix.shape[0]):
            for j in range(main_matrix.shape[1]):
                try:
                    if thisblock[i][0][j] in word_to_ix:
                        main_matrix[i,j] = word_to_ix[thisblock[i][0][j]]
                    
                except IndexError:
                    pass       # because initialze with 0, so you pad with 0
    
        xxx2=[]
        yyy=[]
        for ii in thisblock:
            xxx2.append(ii[1])
            yyy.append(ii[2])
        
        xxx2=np.array(xxx2)
        yyy=np.array(yyy)
        batch_data.append((autograd.Variable(torch.from_numpy(main_matrix)),autograd.Variable(torch.FloatTensor(xxx2)),autograd.Variable(torch.FloatTensor(yyy))))
    return batch_data

In [65]:
batchtraining_data=preprocessing(training_data)
batchtest_data=preprocessing(test_data)
batchval_data=preprocessing(val_data)

In [68]:
######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class CAML(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(CAML, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.embed_drop = nn.Dropout(p=0.2)   
        
        
        self.convs1 = nn.Conv1d(Embeddingsize,300,10,padding=5)
        self.H=nn.Linear(300, tagset_size )   
        self.final = nn.Linear(300, tagset_size)
        
        self.layer2 = nn.Linear(Embeddingsize, 1)
        self.embedding=nn.Linear(rvocsize,Embeddingsize,bias=False)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, vec1, nvec, wiki, simlearning):
        
       
        thisembeddings=self.word_embeddings(vec1)
        thisembeddings = self.embed_drop(thisembeddings)
        thisembeddings=thisembeddings.transpose(1,2)
        
        
        thisembeddings=self.tanh(self.convs1(thisembeddings).transpose(1,2))  
        
        alpha=self.H.weight.matmul(thisembeddings.transpose(1,2))
        alpha=F.softmax(alpha, dim=2)
        
        m=alpha.matmul(thisembeddings)
       
        myfinal=self.final.weight.mul(m).sum(dim=2).add(self.final.bias)
        
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
       
        if simlearning==1:
            tag_scores = self.sigmoid(myfinal.detach()+vec3)
        else:
            tag_scores = self.sigmoid(myfinal)
        
        
        return tag_scores



topk=10

def trainmodel(model, sim):
    print ('start_training')
    modelsaved=[]
    modelperform=[]
    topk=10
    
    
    bestresults=-1
    bestiter=-1
    for epoch in range(5000):  
        model.train()
        
        lossestrain = []
        recall=[]
        for mysentence in batchtraining_data:
            model.zero_grad()
            
            targets = mysentence[2].cuda()
            tag_scores = model(mysentence[0].cuda(),mysentence[1].cuda(),wikivec.cuda(),sim)
            loss = loss_function(tag_scores, targets)
            loss.backward()
            optimizer.step()
            lossestrain.append(loss.data.mean())
        print (epoch)
        modelsaved.append(copy.deepcopy(model.state_dict()))
        print ("XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        model.eval()
    
        recall=[]
        for inputs in batchval_data:
           
            targets = inputs[2].cuda()
            tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)
    
            loss = loss_function(tag_scores, targets)
            
            targets=targets.data.cpu().numpy()
            tag_scores= tag_scores.data.cpu().numpy()
            
            
            for iii in range(0,len(tag_scores)):
                temp={}
                for iiii in range(0,len(tag_scores[iii])):
                    temp[iiii]=tag_scores[iii][iiii]
                temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
                thistop=int(np.sum(targets[iii]))
                hit=0.0
                for ii in temp1[0:max(thistop,topk)]:
                    if targets[iii][ii[0]]==1.0:
                        hit=hit+1
                if thistop!=0:
                    recall.append(hit/thistop)
            
        print ('validation top-',topk, np.mean(recall))
        
        
        
        modelperform.append(np.mean(recall))
        if modelperform[-1]>bestresults:
            bestresults=modelperform[-1]
            bestiter=len(modelperform)-1
        
        if(len(modelperform)-bestiter)>5:
            print (modelperform,bestiter)
            return modelsaved[bestiter]
    
model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()

loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

basemodel= trainmodel(model, 0)
torch.save(basemodel, 'CAML_model')

model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()
model.load_state_dict(basemodel)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
KSImodel= trainmodel(model, 1)
torch.save(KSImodel, 'KSI_CAML_model')

def testmodel(modelstate, sim):
    model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
    model.cuda()
    model.load_state_dict(modelstate)
    loss_function = nn.BCELoss()
    model.eval()
    recall=[]
    lossestest = []
    
    y_true=[]
    y_scores=[]
    
    
    for inputs in batchtest_data:
       
        targets = inputs[2].cuda()
        
        tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)

        loss = loss_function(tag_scores, targets)
        
        targets=targets.data.cpu().numpy()
        tag_scores= tag_scores.data.cpu().numpy()
        
        
        lossestest.append(loss.data.mean())
        y_true.append(targets)
        y_scores.append(tag_scores)
        
        for iii in range(0,len(tag_scores)):
            temp={}
            for iiii in range(0,len(tag_scores[iii])):
                temp[iiii]=tag_scores[iii][iiii]
            temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
            thistop=int(np.sum(targets[iii]))
            hit=0.0
            
            for ii in temp1[0:max(thistop,topk)]:
                if targets[iii][ii[0]]==1.0:
                    hit=hit+1
            if thistop!=0:
                recall.append(hit/thistop)
    y_true=np.concatenate(y_true,axis=0)
    y_scores=np.concatenate(y_scores,axis=0)
    y_true=y_true.T
    y_scores=y_scores.T
    temptrue=[]
    tempscores=[]
    for  col in range(0,len(y_true)):
        if np.sum(y_true[col])!=0:
            temptrue.append(y_true[col])
            tempscores.append(y_scores[col])
    temptrue=np.array(temptrue)
    tempscores=np.array(tempscores)
    y_true=temptrue.T
    y_scores=tempscores.T
    y_pred=(y_scores>0.5).astype(np.int32)
    lossarray = []
    
    for i in lossestest:
        lossarray.append(i.item())
    print ('test loss', np.mean(lossarray))
    print ('top-',topk, np.mean(recall))
    print ('macro AUC', roc_auc_score(y_true, y_scores,average='macro'))
    print ('micro AUC', roc_auc_score(y_true, y_scores,average='micro'))
    print ('macro F1', f1_score(y_true, y_pred, average='macro')  )
    print ('micro F1', f1_score(y_true, y_pred, average='micro')  )

print ('CAML alone:           ')
testmodel(basemodel, 0)
print ('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print ('KSI+CAML:           ')
testmodel(KSImodel, 1)


start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.2111111111111111
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
4
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
5
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
6
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
[0.2111111111111111, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556] 1
start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
4
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
5
X

**KSI_CNN**

In [40]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import copy

##########################################################

label_to_ix=np.load('label_to_ix.npy',allow_pickle=True).item()
ix_to_label=np.load('ix_to_label.npy',allow_pickle=True)
training_data=np.load('training_data.npy',allow_pickle=True)
test_data=np.load('test_data.npy',allow_pickle=True)
val_data=np.load('val_data.npy',allow_pickle=True)
word_to_ix=np.load('word_to_ix.npy',allow_pickle=True).item()
ix_to_word=np.load('ix_to_word.npy',allow_pickle=True)
newwikivec=np.load('newwikivec.npy',allow_pickle=True)
wikivoc=np.load('wikivoc.npy',allow_pickle=True).item()



wikisize=newwikivec.shape[0]
rvocsize=newwikivec.shape[1]
wikivec=autograd.Variable(torch.FloatTensor(newwikivec))

batchsize=8


def preprocessing(data):

    new_data=[]
    for i, note, j in data:
        templabel=[0.0]*len(label_to_ix)
        for jj in j:
            if jj in wikivoc:
                templabel[label_to_ix[jj]]=1.0
        templabel=np.array(templabel,dtype=float)
        new_data.append((i, note, templabel))
    new_data=np.array(new_data,dtype=object)
    
    lenlist=[]
    for i in new_data:
        lenlist.append(len(i[0]))
    sortlen=sorted(range(len(lenlist)), key=lambda k: lenlist[k])  
    new_data=new_data[sortlen]

    batch_data=[]
    
    for start_ix in range(0, len(new_data)-batchsize+1, batchsize):
        thisblock=new_data[start_ix:start_ix+batchsize]
        mybsize= len(thisblock)
        numword=np.max([len(ii[0]) for ii in thisblock])
        main_matrix = np.zeros((mybsize, numword), dtype= np.int32)
        for i in range(main_matrix.shape[0]):
            for j in range(main_matrix.shape[1]):
                try:
                    if thisblock[i][0][j] in word_to_ix:
                        main_matrix[i,j] = word_to_ix[thisblock[i][0][j]]
                    
                except IndexError:
                    pass       # because initialze with 0, so you pad with 0
    
        xxx2=[]
        yyy=[]
        for ii in thisblock:
            xxx2.append(ii[1])
            yyy.append(ii[2])
        
        xxx2=np.array(xxx2)
        yyy=np.array(yyy)
        batch_data.append((autograd.Variable(torch.from_numpy(main_matrix)),autograd.Variable(torch.FloatTensor(xxx2)),autograd.Variable(torch.FloatTensor(yyy))))
        
    return batch_data

batchtraining_data=preprocessing(training_data)
batchtest_data=preprocessing(test_data)
batchval_data=preprocessing(val_data)


In [62]:


######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class CNN(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(CNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.embed_drop = nn.Dropout(p=0.2)
        
        self.hidden2tag = nn.Linear(300, tagset_size)
        
        
        self.convs1 = nn.Conv1d(Embeddingsize,100,3)
        self.convs2 = nn.Conv1d(Embeddingsize,100,4)
        self.convs3 = nn.Conv1d(Embeddingsize,100,5)
        
        
        self.layer2 = nn.Linear(Embeddingsize, 1,bias=False)
        self.embedding=nn.Linear(rvocsize,Embeddingsize)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, vec1, nvec, wiki, simlearning):
       
        thisembeddings=self.word_embeddings(vec1)
        thisembeddings = self.embed_drop(thisembeddings)
        thisembeddings=thisembeddings.transpose(1,2)
        
        output1=self.tanh(self.convs1(thisembeddings))
        output1=nn.MaxPool1d(output1.size()[2])(output1)
        
        output2=self.tanh(self.convs2(thisembeddings))
        output2=nn.MaxPool1d(output2.size()[2])(output2)
        
        output3=self.tanh(self.convs3(thisembeddings))
        output3=nn.MaxPool1d(output3.size()[2])(output3)
        
        output4 = torch.cat([output1,output2,output3], 1).squeeze(2)
        
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
       
        vec2 = self.hidden2tag(output4)
        if simlearning==1:
            tag_scores = self.sigmoid(vec2.detach()+vec3)
        else:
            tag_scores = self.sigmoid(vec2)
        
        
        return tag_scores

######################################################################
# Train the model:

topk=10

def trainmodel(model, sim):
    print ('start_training')
    modelsaved=[]
    modelperform=[]
    topk=10
    
    
    bestresults=-1
    bestiter=-1
    for epoch in range(5000):  
        model.train()
        
        lossestrain = []
        recall=[]
        for mysentence in batchtraining_data:
            model.zero_grad()
            
            targets = mysentence[2].cuda()
            tag_scores = model(mysentence[0].cuda(),mysentence[1].cuda(),wikivec.cuda(),sim)
            loss = loss_function(tag_scores, targets)
            loss.backward()
            optimizer.step()
            lossestrain.append(loss.data.mean())
        print (epoch)
        modelsaved.append(copy.deepcopy(model.state_dict()))
        print ("XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        model.eval()
    
        recall=[]
        for inputs in batchval_data:
           
            targets = inputs[2].cuda()
            tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)
    
            loss = loss_function(tag_scores, targets)
            
            targets=targets.data.cpu().numpy()
            tag_scores= tag_scores.data.cpu().numpy()
            
            
            for iii in range(0,len(tag_scores)):
                temp={}
                for iiii in range(0,len(tag_scores[iii])):
                    temp[iiii]=tag_scores[iii][iiii]
                temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
                thistop=int(np.sum(targets[iii]))
                hit=0.0
                for ii in temp1[0:max(thistop,topk)]:
                    if targets[iii][ii[0]]==1.0:
                        hit=hit+1
                if thistop!=0:
                    recall.append(hit/thistop)
            
        print ('validation top-',topk, np.mean(recall))
        
        
        
        modelperform.append(np.mean(recall))
        if modelperform[-1]>bestresults:
            bestresults=modelperform[-1]
            bestiter=len(modelperform)-1
        
        if (len(modelperform)-bestiter)>5:
            print (modelperform,bestiter)
            return modelsaved[bestiter]
    
model = CNN(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()

loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

basemodel= trainmodel(model, 0)
torch.save(basemodel, 'CNN_model')

model = CNN(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()
model.load_state_dict(basemodel)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
KSImodel= trainmodel(model, 1)
torch.save(KSImodel, 'KSI_CNN_model')

def testmodel(modelstate, sim):
    model = CNN(batchsize, len(word_to_ix), len(label_to_ix))
    model.cuda()
    model.load_state_dict(modelstate)
    loss_function = nn.BCELoss()
    model.eval()
    recall = []
    lossestest = []
    
    y_true = []  # Initialize as empty list
    y_scores = []  # Initialize as empty list
   
    for inputs in batchtest_data:
        
        targets = inputs[2].cuda()
        tag_scores = model(inputs[0].cuda(), inputs[1].cuda(), wikivec.cuda(), sim)

        loss = loss_function(tag_scores, targets)
        
        targets = targets.data.cpu().numpy()
        tag_scores = tag_scores.data.cpu().numpy()
        
        lossestest.append(loss.data.mean())
        y_true.append(targets)
        y_scores.append(tag_scores)
        for iii in range(0, len(tag_scores)):
            temp = {}
            for iiii in range(0, len(tag_scores[iii])):
                temp[iiii] = tag_scores[iii][iiii]
            temp1 = [(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
            thistop = int(np.sum(targets[iii]))
            hit = 0.0
            
            for ii in temp1[0:max(thistop, topk)]:
                if targets[iii][ii[0]] == 1.0:
                    hit = hit + 1
            if thistop != 0:
                recall.append(hit / thistop)
    y_true = np.concatenate(y_true, axis=0)
    y_scores = np.concatenate(y_scores, axis=0)

    y_true = y_true.T
    y_scores = y_scores.T
    temptrue = []
    tempscores = []
    for col in range(0, len(y_true)):
        if np.sum(y_true[col]) != 0:
            temptrue.append(y_true[col])
            tempscores.append(y_scores[col])
    temptrue = np.array(temptrue)
    tempscores = np.array(tempscores)
    y_true = temptrue.T
    y_scores = tempscores.T
    y_pred = (y_scores > 0.5).astype(np.int32)
    lossarray = []
    
    for i in lossestest:
        lossarray.append(i.item())
        
    print('test loss', np.mean(lossarray))
    print('top-', topk, np.mean(recall))
    print('macro AUC', roc_auc_score(y_true, y_scores, average='macro'))
    print('micro AUC', roc_auc_score(y_true, y_scores, average='micro'))
    print('macro F1', f1_score(y_true, y_pred, average='macro'))
    print('micro F1', f1_score(y_true, y_pred, average='micro'))

print ('CNN alone:           ')
testmodel(basemodel, 0)
print ('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print ('KSI+CNN:           ')
testmodel(KSImodel, 1)

start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.14444444444444446
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
4
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
5
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
6
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
[0.14444444444444446, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556] 1
start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
4
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111


**KSI_LSTM**

In [70]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import copy

##########################################################

label_to_ix=np.load('label_to_ix.npy',allow_pickle=True).item()
ix_to_label=np.load('ix_to_label.npy',allow_pickle=True)
training_data=np.load('training_data.npy',allow_pickle=True)
test_data=np.load('test_data.npy',allow_pickle=True)
val_data=np.load('val_data.npy',allow_pickle=True)
word_to_ix=np.load('word_to_ix.npy',allow_pickle=True).item()
ix_to_word=np.load('ix_to_word.npy',allow_pickle=True)
newwikivec=np.load('newwikivec.npy',allow_pickle=True)
wikivoc=np.load('wikivoc.npy',allow_pickle=True).item()

wikisize=newwikivec.shape[0]
rvocsize=newwikivec.shape[1]
wikivec=autograd.Variable(torch.FloatTensor(newwikivec))


batchsize=8



def preprocessing(data):

    new_data=[]
    for i, note, j in data:
        templabel=[0.0]*len(label_to_ix)
        for jj in j:
            if jj in wikivoc:
                templabel[label_to_ix[jj]]=1.0
        templabel=np.array(templabel,dtype=float)
        new_data.append((i, note, templabel))
    new_data=np.array(new_data)
    
    lenlist=[]
    for i in new_data:
        lenlist.append(len(i[0]))
    sortlen=sorted(range(len(lenlist)), key=lambda k: lenlist[k])  
    new_data=new_data[sortlen]
    
    batch_data=[]
    
    for start_ix in range(0, len(new_data)-batchsize+1, batchsize):
        thisblock=new_data[start_ix:start_ix+batchsize]
        mybsize= len(thisblock)
        numword=np.max([len(ii[0]) for ii in thisblock])
        main_matrix = np.zeros((mybsize, numword), dtype= np.int)
        for i in range(main_matrix.shape[0]):
            for j in range(main_matrix.shape[1]):
                try:
                    if thisblock[i][0][j] in word_to_ix:
                        main_matrix[i,j] = word_to_ix[thisblock[i][0][j]]
                    
                except IndexError:
                    pass       # because initialze with 0, so you pad with 0
    
        xxx2=[]
        yyy=[]
        for ii in thisblock:
            xxx2.append(ii[1])
            yyy.append(ii[2])
        
        xxx2=np.array(xxx2)
        yyy=np.array(yyy)
        batch_data.append((autograd.Variable(torch.from_numpy(main_matrix)),autograd.Variable(torch.FloatTensor(xxx2)),autograd.Variable(torch.FloatTensor(yyy))))
    return batch_data
batchtraining_data=preprocessing(training_data)
batchtest_data=preprocessing(test_data)
batchval_data=preprocessing(val_data)


######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class LSTM(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.lstm = nn.LSTM(Embeddingsize, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
        
        self.layer2 = nn.Linear(Embeddingsize, 1,bias=False)
        self.embedding=nn.Linear(rvocsize,Embeddingsize)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize,bias=False)
        
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.embed_drop = nn.Dropout(p=0.2)
    
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim)).cuda())

    
    def forward(self, vec1, nvec, wiki, simlearning):
      
        thisembeddings=self.word_embeddings(vec1).transpose(0,1)
        thisembeddings = self.embed_drop(thisembeddings)
       
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
        
        
        lstm_out, self.hidden = self.lstm(
            thisembeddings, self.hidden)
        
        lstm_out=lstm_out.transpose(0,2).transpose(0,1)
        
        output1=nn.MaxPool1d(lstm_out.size()[2])(lstm_out).view(batchsize,-1)
        
        vec2 = self.hidden2tag(output1)
        if simlearning==1:
            tag_scores = self.sigmoid(vec2.detach()+vec3)
        else:
            tag_scores = self.sigmoid(vec2)
        
        
        return tag_scores

######################################################################
# Train the model:

topk=10

def trainmodel(model, sim):
    print ('start_training')
    modelsaved=[]
    modelperform=[]
    topk=10
    
    
    bestresults=-1
    bestiter=-1
    for epoch in range(5000):  
        
        model.train()
        
        lossestrain = []
        recall=[]
        for mysentence in batchtraining_data:
            model.zero_grad()
            model.hidden = model.init_hidden()
            targets = mysentence[2].cuda()
            tag_scores = model(mysentence[0].cuda(),mysentence[1].cuda(),wikivec.cuda(),sim)
            loss = loss_function(tag_scores, targets)
            loss.backward()
            optimizer.step()
            lossestrain.append(loss.data.mean())
        print (epoch)
        
        modelsaved.append(copy.deepcopy(model.state_dict()))
        print ("XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        model.eval()
    
        recall=[]
        for inputs in batchval_data:
            model.hidden = model.init_hidden()
            targets = inputs[2].cuda()
            tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)
    
            loss = loss_function(tag_scores, targets)
            
            targets=targets.data.cpu().numpy()
            tag_scores= tag_scores.data.cpu().numpy()
            
            
            for iii in range(0,len(tag_scores)):
                temp={}
                for iiii in range(0,len(tag_scores[iii])):
                    temp[iiii]=tag_scores[iii][iiii]
                temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
                thistop=int(np.sum(targets[iii]))
                hit=0.0
                for ii in temp1[0:max(thistop,topk)]:
                    if targets[iii][ii[0]]==1.0:
                        hit=hit+1
                if thistop!=0:
                    recall.append(hit/thistop)
            
        print ('validation top-',topk, np.mean(recall))
        
        
        
        modelperform.append(np.mean(recall))
        if modelperform[-1]>bestresults:
            bestresults=modelperform[-1]
            bestiter=len(modelperform)-1
        
        if (len(modelperform)-bestiter)>5:
            print (modelperform,bestiter)
            return modelsaved[bestiter]
    
model = LSTM(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()

loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

basemodel= trainmodel(model, 0)
torch.save(basemodel, 'LSTM_model')

model = LSTM(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()
model.load_state_dict(basemodel)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
KSImodel= trainmodel(model, 1)
torch.save(KSImodel, 'KSI_LSTM_model')

def testmodel(modelstate, sim):
    model = LSTM(batchsize, len(word_to_ix), len(label_to_ix))
    model.cuda()
    model.load_state_dict(modelstate)
    loss_function = nn.BCELoss()
    model.eval()
    recall=[]
    lossestest = []
    
    y_true=[]
    y_scores=[]
    
    
    for inputs in batchtest_data:
        model.hidden = model.init_hidden()
        targets = inputs[2].cuda()
        
        tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)

        loss = loss_function(tag_scores, targets)
        
        targets=targets.data.cpu().numpy()
        tag_scores= tag_scores.data.cpu().numpy()
        
        
        lossestest.append(loss.data.mean())
        y_true.append(targets)
        y_scores.append(tag_scores)
        
        for iii in range(0,len(tag_scores)):
            temp={}
            for iiii in range(0,len(tag_scores[iii])):
                temp[iiii]=tag_scores[iii][iiii]
            temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
            thistop=int(np.sum(targets[iii]))
            hit=0.0
            
            for ii in temp1[0:max(thistop,topk)]:
                if targets[iii][ii[0]]==1.0:
                    hit=hit+1
            if thistop!=0:
                recall.append(hit/thistop)
    y_true=np.concatenate(y_true,axis=0)
    y_scores=np.concatenate(y_scores,axis=0)
    y_true=y_true.T
    y_scores=y_scores.T
    temptrue=[]
    tempscores=[]
    for  col in range(0,len(y_true)):
        if np.sum(y_true[col])!=0:
            temptrue.append(y_true[col])
            tempscores.append(y_scores[col])
    temptrue=np.array(temptrue)
    tempscores=np.array(tempscores)
    y_true=temptrue.T
    y_scores=tempscores.T
    y_pred=(y_scores>0.5).astype(np.int)
    lossarray = []
    for i in lossestest:
        lossarray.append(i.item())
        
    print('test loss', np.mean(lossarray))
    print ('top-',topk, np.mean(recall))
    print ('macro AUC', roc_auc_score(y_true, y_scores,average='macro'))
    print ('micro AUC', roc_auc_score(y_true, y_scores,average='micro'))
    print ('macro F1', f1_score(y_true, y_pred, average='macro')  )
    print ('micro F1', f1_score(y_true, y_pred, average='micro')  )

print ('LSTM alone:           ')
testmodel(basemodel, 0)
print ('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print ('KSI+LSTM:           ')
testmodel(KSImodel, 1)

<ipython-input-70-fa94dd879dd3>:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_data=np.array(new_data)
<ipython-input-70-fa94dd879dd3>:56: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  main_matrix = np.zeros((mybsize, numword), dtype= np.int)


start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.4888888888888889
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.43333333333333335
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
4
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
5
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
6
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
7
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
[0.4888888888888889, 0.43333333333333335, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556] 2
start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 

<ipython-input-70-fa94dd879dd3>:292: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred=(y_scores>0.5).astype(np.int)
<ipython-input-70-fa94dd879dd3>:292: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20;

**KSI_LSTMatt**

In [71]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import copy

##########################################################

label_to_ix=np.load('label_to_ix.npy',allow_pickle=True).item()
ix_to_label=np.load('ix_to_label.npy',allow_pickle=True)
training_data=np.load('training_data.npy',allow_pickle=True)
test_data=np.load('test_data.npy',allow_pickle=True)
val_data=np.load('val_data.npy',allow_pickle=True)
word_to_ix=np.load('word_to_ix.npy',allow_pickle=True).item()
ix_to_word=np.load('ix_to_word.npy',allow_pickle=True)
newwikivec=np.load('newwikivec.npy',allow_pickle=True)
wikivoc=np.load('wikivoc.npy',allow_pickle=True).item()

wikisize=newwikivec.shape[0]
rvocsize=newwikivec.shape[1]
wikivec=autograd.Variable(torch.FloatTensor(newwikivec))

batchsize=8



def preprocessing(data):

    new_data=[]
    for i, note, j in data:
        templabel=[0.0]*len(label_to_ix)
        for jj in j:
            if jj in wikivoc:
                templabel[label_to_ix[jj]]=1.0
        templabel=np.array(templabel,dtype=float)
        new_data.append((i, note, templabel))
    new_data=np.array(new_data)
    
    lenlist=[]
    for i in new_data:
        lenlist.append(len(i[0]))
    sortlen=sorted(range(len(lenlist)), key=lambda k: lenlist[k])  
    new_data=new_data[sortlen]
    
    batch_data=[]
    
    for start_ix in range(0, len(new_data)-batchsize+1, batchsize):
        thisblock=new_data[start_ix:start_ix+batchsize]
        mybsize= len(thisblock)
        numword=np.max([len(ii[0]) for ii in thisblock])
        main_matrix = np.zeros((mybsize, numword), dtype= np.int)
        for i in range(main_matrix.shape[0]):
            for j in range(main_matrix.shape[1]):
                try:
                    if thisblock[i][0][j] in word_to_ix:
                        main_matrix[i,j] = word_to_ix[thisblock[i][0][j]]
                    
                except IndexError:
                    pass       # because initialze with 0, so you pad with 0
    
        xxx2=[]
        yyy=[]
        for ii in thisblock:
            xxx2.append(ii[1])
            yyy.append(ii[2])
        
        xxx2=np.array(xxx2)
        yyy=np.array(yyy)
        batch_data.append((autograd.Variable(torch.from_numpy(main_matrix)),autograd.Variable(torch.FloatTensor(xxx2)),autograd.Variable(torch.FloatTensor(yyy))))
    return batch_data
batchtraining_data=preprocessing(training_data)
batchtest_data=preprocessing(test_data)
batchval_data=preprocessing(val_data)



######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class LSTMattn(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(LSTMattn, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.lstm = nn.LSTM(Embeddingsize, hidden_dim)
        self.hidden = self.init_hidden()
        
        self.H=nn.Linear(hidden_dim, tagset_size )  
        self.final = nn.Linear(hidden_dim, tagset_size)
        
        self.layer2 = nn.Linear(Embeddingsize, 1,bias=False)
        self.embedding=nn.Linear(rvocsize,Embeddingsize)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize,bias=False)
        
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.embed_drop = nn.Dropout(p=0.2)
    
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim)).cuda())

    
    def forward(self, vec1, nvec, wiki, simlearning):
        
        
        thisembeddings=self.word_embeddings(vec1).transpose(0,1)
        thisembeddings = self.embed_drop(thisembeddings)
        
        
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
        
        lstm_out, self.hidden = self.lstm(
            thisembeddings, self.hidden)
        
        
        
        lstm_out=lstm_out.transpose(0,1)

        alpha=self.H.weight.matmul(lstm_out.transpose(1,2))
        alpha=F.softmax(alpha, dim=2)
        
        m=alpha.matmul(lstm_out)
        
        myfinal=self.final.weight.mul(m).sum(dim=2).add(self.final.bias)
        
        
        if simlearning==1:
            tag_scores = self.sigmoid(myfinal.detach()+vec3)
        else:
            tag_scores = self.sigmoid(myfinal)
        
        
        return tag_scores

######################################################################
# Train the model:

topk=10

def trainmodel(model, sim):
    print ('start_training')
    modelsaved=[]
    modelperform=[]
    topk=10
    
    
    bestresults=-1
    bestiter=-1
    for epoch in range(5000):  
       
        model.train()
        
        lossestrain = []
        recall=[]
        for mysentence in batchtraining_data:
            model.zero_grad()
            model.hidden = model.init_hidden()
            targets = mysentence[2].cuda()
            tag_scores = model(mysentence[0].cuda(),mysentence[1].cuda(),wikivec.cuda(),sim)
            loss = loss_function(tag_scores, targets)
            loss.backward()
            optimizer.step()
            lossestrain.append(loss.data.mean())
        print (epoch)
        modelsaved.append(copy.deepcopy(model.state_dict()))
        print ("XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        model.eval()
    
        recall=[]
        for inputs in batchval_data:
            model.hidden = model.init_hidden()
            targets = inputs[2].cuda()
            tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)
    
            loss = loss_function(tag_scores, targets)
            
            targets=targets.data.cpu().numpy()
            tag_scores= tag_scores.data.cpu().numpy()
            
            
            for iii in range(0,len(tag_scores)):
                temp={}
                for iiii in range(0,len(tag_scores[iii])):
                    temp[iiii]=tag_scores[iii][iiii]
                temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
                thistop=int(np.sum(targets[iii]))
                hit=0.0
                for ii in temp1[0:max(thistop,topk)]:
                    if targets[iii][ii[0]]==1.0:
                        hit=hit+1
                if thistop!=0:
                    recall.append(hit/thistop)
            
        print ('validation top-',topk, np.mean(recall))
        
        
        
        modelperform.append(np.mean(recall))
        if modelperform[-1]>bestresults:
            bestresults=modelperform[-1]
            bestiter=len(modelperform)-1
        
        if (len(modelperform)-bestiter)>5:
            print (modelperform,bestiter)
            return modelsaved[bestiter]
    
model = LSTMattn(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

basemodel= trainmodel(model, 0)
torch.save(basemodel, 'LSTMattn_model')

model = LSTMattn(batchsize, len(word_to_ix), len(label_to_ix))
model.cuda()
model.load_state_dict(basemodel)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
KSImodel= trainmodel(model, 1)
torch.save(KSImodel, 'KSI_LSTMattn_model')

def testmodel(modelstate, sim):
    model = LSTMattn(batchsize, len(word_to_ix), len(label_to_ix))
    model.cuda()
    model.load_state_dict(modelstate)
    loss_function = nn.BCELoss()
    model.eval()
    recall=[]
    lossestest = []
    
    y_true=[]
    y_scores=[]
    
    
    for inputs in batchtest_data:
        model.hidden = model.init_hidden()
        targets = inputs[2].cuda()
        
        tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)

        loss = loss_function(tag_scores, targets)
        
        targets=targets.data.cpu().numpy()
        tag_scores= tag_scores.data.cpu().numpy()
        
        
        lossestest.append(loss.data.mean())
        y_true.append(targets)
        y_scores.append(tag_scores)
        
        for iii in range(0,len(tag_scores)):
            temp={}
            for iiii in range(0,len(tag_scores[iii])):
                temp[iiii]=tag_scores[iii][iiii]
            temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
            thistop=int(np.sum(targets[iii]))
            hit=0.0
            
            for ii in temp1[0:max(thistop,topk)]:
                if targets[iii][ii[0]]==1.0:
                    hit=hit+1
            if thistop!=0:
                recall.append(hit/thistop)
    y_true=np.concatenate(y_true,axis=0)
    y_scores=np.concatenate(y_scores,axis=0)
    y_true=y_true.T
    y_scores=y_scores.T
    temptrue=[]
    tempscores=[]
    for  col in range(0,len(y_true)):
        if np.sum(y_true[col])!=0:
            temptrue.append(y_true[col])
            tempscores.append(y_scores[col])
    temptrue=np.array(temptrue)
    tempscores=np.array(tempscores)
    y_true=temptrue.T
    y_scores=tempscores.T
    y_pred=(y_scores>0.5).astype(np.int)
    lossarray = []
    for i in lossestest:
        lossarray.append(i.item())
        
    print('test loss', np.mean(lossarray))
    print ('top-',topk, np.mean(recall))
    print ('macro AUC', roc_auc_score(y_true, y_scores,average='macro'))
    print ('micro AUC', roc_auc_score(y_true, y_scores,average='micro'))
    print ('macro F1', f1_score(y_true, y_pred, average='macro')  )
    print ('micro F1', f1_score(y_true, y_pred, average='micro')  )

print ('LSTMattn alone:           ')
testmodel(basemodel, 0)
print ('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print ('KSI+LSTMattn:           ')
testmodel(KSImodel, 1)

<ipython-input-71-7b7b527c1884>:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_data=np.array(new_data)
<ipython-input-71-7b7b527c1884>:56: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  main_matrix = np.zeros((mybsize, numword), dtype= np.int)


start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.17777777777777778
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
4
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
5
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
6
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
7
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
[0.17777777777777778, 0.5, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556, 0.5555555555555556] 2
start_training
0
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.5555555555555556
1
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
2
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
3
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
validation top- 10 0.611111111111111
4
XXXXXXXX

<ipython-input-71-7b7b527c1884>:299: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred=(y_scores>0.5).astype(np.int)
<ipython-input-71-7b7b527c1884>:299: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20;